<a href="https://colab.research.google.com/github/PrathibhaMangali/TwitterSentimentalAnaysis/blob/main/AIMLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"riturajdutta","key":"f51cb1396a1eaa532cef82ba012f8118"}'}

In [ ]:
! mkdir -p ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! mkdir -p ~/.kaggle

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 74% 60.0M/80.9M [00:00<00:00, 197MB/s]
100% 80.9M/80.9M [00:00<00:00, 204MB/s]


In [ ]:
from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
   zip.extractall()
   print('The dataset, is extracted')

The dataset, is extracted


In [ ]:
import nltk

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')


In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
column_names=['target','ids','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',names=column_names)


In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
twitter_data.target.value_counts()

,count
target,
0,800000
4,800000


In [ ]:
twitter_data.replace({'target':{4:1}},inplace=True)


In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
from nltk.stem import PorterStemmer

# Initialize stemmer
stemmer = PorterStemmer()

# Vectorized stemming using list comprehension
twitter_data['stemmed_content'] = [" ".join([stemmer.stem(word) for word in text.split()]) for text in twitter_data['text']]


In [ ]:
 twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can't updat hi facebook by te...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichan i dive mani time for the ball. manag...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, it' not behav at all. i'm..."


In [ ]:
print(twitter_data['stemmed_content'])

0          @switchfoot http://twitpic.com/2y1zl - awww, t...
1          is upset that he can't updat hi facebook by te...
2          @kenichan i dive mani time for the ball. manag...
3               my whole bodi feel itchi and like it on fire
4          @nationwideclass no, it' not behav at all. i'm...
                                 ...                        
1599995    just woke up. have no school is the best feel ...
1599996    thewdb.com - veri cool to hear old walt interv...
1599997    are you readi for your mojo makeover? ask me f...
1599998    happi 38th birthday to my boo of alll time!!! ...
1599999    happi #charitytuesday @thenspcc @sparkschar @s...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
print(X)

["@switchfoot http://twitpic.com/2y1zl - awww, that' a bummer. you shoulda got david carr of third day to do it. ;d"
 "is upset that he can't updat hi facebook by text it... and might cri as a result school today also. blah!"
 '@kenichan i dive mani time for the ball. manag to save 50% the rest go out of bound'
 ... 'are you readi for your mojo makeover? ask me for detail'
 'happi 38th birthday to my boo of alll time!!! tupac amaru shakur'
 'happi #charitytuesday @thenspcc @sparkschar @speakinguph4h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['about to watch saw iv and drink a lil wine' "@hatermagazin i'm in!"
 'even though it my favourit drink i think it the vodka and coke that wipe my mind all the time think im gonna have to find a new drink'
 ... 'is eager for monday afternoon'
 "hope everyon and their mother had a great day! can't wait to hear what the guy have in store tomorrow!"
 'i love wake up to folgers. too bad my voic wa deeper than his.']


In [ ]:
print(X_test)

["@mmangen - m do fine! i haven't had much time to chat on twitter. hubbi is back for the summer &amp; tend to domin my free time."
 'at ah may show w/ ruth kim &amp; geoffrey sanhueza'
 '@ishatara mayb it wa onli a bay area thang dammit!' ...
 '@destini41 nevertheless hooray! for 4700 member and have a wonder and safe trip!'
 'not feel too well' '@supersandro !!!! thank you!']


In [ ]:
vectorizer=TfidfVectorizer()

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [ ]:
print(X_train)

  (0, 40279)	0.24921248655385633
  (0, 519169)	0.1251682928604393
  (0, 548737)	0.2582573574441168
  (0, 453119)	0.3393001361168507
  (0, 254267)	0.4975163889508718
  (0, 56947)	0.1541598714398116
  (0, 166830)	0.3586993165413134
  (0, 311959)	0.39943217862098934
  (0, 555970)	0.42532987400715944
  (1, 226262)	0.9765112282750745
  (1, 247278)	0.21546651956326146
  (2, 519169)	0.07448645608930922
  (2, 56947)	0.09173906771691974
  (2, 166830)	0.426917077483995
  (2, 184426)	0.17609618428598842
  (2, 514641)	0.17213679575154026
  (2, 252843)	0.17765676072426673
  (2, 366485)	0.1815125237069395
  (2, 190522)	0.27019459809118346
  (2, 513658)	0.2962743072556722
  (2, 507873)	0.15389117177429465
  (2, 545131)	0.3034883248851407
  (2, 129252)	0.28852240010234204
  (2, 507481)	0.10868332108979102
  (2, 556379)	0.3110561917625645
  :	:
  (1279998, 148756)	0.18288427025358445
  (1279998, 222374)	0.2167702817289201
  (1279998, 236205)	0.2214524643637042
  (1279998, 552329)	0.20454757140378815
  

In [ ]:
print(X_test)

  (0, 55622)	0.1563041688840363
  (0, 74691)	0.14771930001474184
  (0, 117476)	0.24481786746646433
  (0, 161994)	0.1364625632059021
  (0, 163021)	0.3427790151715253
  (0, 193829)	0.2323190566716047
  (0, 198156)	0.10402819171551929
  (0, 200572)	0.21642485492921726
  (0, 222374)	0.15352997153281683
  (0, 226531)	0.21181176814513514
  (0, 238369)	0.2631863878189709
  (0, 251929)	0.10105303919622358
  (0, 354510)	0.4122535104060292
  (0, 364066)	0.1620496343375773
  (0, 366485)	0.09288398012183557
  (0, 390585)	0.11211826947842243
  (0, 492694)	0.20033417376282855
  (0, 505261)	0.31730648524047184
  (0, 507873)	0.07874952233647493
  (0, 516781)	0.28905256268144397
  (0, 519169)	0.1524654798567129
  (0, 530652)	0.1640664064123227
  (1, 45607)	0.2997439641050589
  (1, 55622)	0.20530566848018322
  (1, 68976)	0.16504622336360802
  :	:
  (319995, 526797)	0.3775238287447342
  (319995, 530652)	0.1921554394629748
  (319995, 557998)	0.27061152642597003
  (319996, 44865)	0.301241362028454
  (31999

In [ ]:
model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [ ]:
print('Accuracy score of the training data: ',training_data_accuracy)

Accuracy score of the training data:  0.8192703125


In [ ]:
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [ ]:
print('Accuracy score of the test data: ',test_data_accuracy)

Accuracy score of the test data:  0.79714375


In [ ]:
import pickle


In [ ]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
X_new=X_test[20]
print(Y_test[20])
prediction=loaded_model.predict(X_new)
print(prediction)
if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

1
[1]
Positive Tweet


In [ ]:
X_new=X_test[11]
print(Y_test[11])
prediction=loaded_model.predict(X_new)
print(prediction)
if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[0]
Negative Tweet
